In [1]:
import re, json

from typing import Union, Any
from math import ceil

import evaluate
import torch as tt
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import PreTrainedModel, PreTrainedTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
from tqdm import tqdm_notebook

In [2]:
# models:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("RuGPT3-RuRACE/checkpoint-65760").to(tt.device("cuda:0"))
tokenizer.pad_token = tokenizer.eos_token

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
with open("tf_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    tf_dataset = json.load(inp)

tf_dataset_train, tf_dataset_val, tf_dataset_test = tf_dataset["train"], tf_dataset["val"], tf_dataset["test"]
tf_dataset_train = Dataset.from_list(tf_dataset_train)
tf_dataset_val = Dataset.from_list(tf_dataset_val)
tf_dataset_test = Dataset.from_list(tf_dataset_test)

option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]

  qtext_orig = example["question"].lower()
  outp = ""

  if ("not true" in qtext_orig) or ("false" in qtext_orig) or ("n't true" in qtext_orig) or ("untrue" in qtext_orig):
    if ("not false" in qtext_orig) or ("n't false" in qtext_orig):
      outp += example['article_ru'] + "\n" + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "
    else:
      outp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту? "
  else:
      outp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "

  outp += f"ПРАВИЛЬНЫЙ ОТВЕТ: {right_answer}"
  outp += "\nНЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:"

  inp = outp

  distractors = ''
  for option in example["options_ru"]:
      if option != right_answer:
          #print(option)
          outp += f"\n  {option}"
          distractors += f"\n  {option}"

  # print(distractors)
  distractors_len = len(tokenizer(distractors)["input_ids"])
  # print(distractors_len)
  #print(outp)
  #raise Exception
  return {"inp": inp, "distractors_len": distractors_len, "outp_expected": outp, "distractors": distractors,"right_answer": right_answer}

tf_dataset_train = tf_dataset_train.map(to_new_format)
tf_dataset_val = tf_dataset_val.map(to_new_format)
tf_dataset_test = tf_dataset_test.map(to_new_format)

Map:   0%|          | 0/3288 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [4]:
tf_dataset_test["distractors"][0]

'\n  Самые великие земледелцы пустыни - это люди.\n  Пустыни быстро растут.\n  Размеры пустыни постоянно меняются.'

In [5]:
distractors_len = pd.Series(tf_dataset_train["distractors_len"])
distractors_len.describe()

count    3288.000000
mean       42.045012
std         9.755723
min        19.000000
25%        35.000000
50%        41.000000
75%        48.000000
max       101.000000
dtype: float64

In [6]:
MAX_OUTPUT_LENGTH = distractors_len.quantile(0.99)
MAX_OUTPUT_LENGTH

69.0

In [7]:
def cut_last_break(input_: list[str]) -> list[str]:
    output = [s[:s.rfind('\n')] for s in input_]
    return output

def parse_options(input_: list[str]) -> list[str]:
    output = [s.strip() for s in input_]
    output = [set(option.strip() for option in s.split('\n')) for s in output]
    output = [sorted(list(s))[:3] for s in output]
    output = ['\n'.join(s) for s in output]
    return output

def get_metric_inputs(
    input_batch: list[str], label_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:

    input_batch_ = tokenizer(input_batch, return_tensors="pt", padding=True)["input_ids"].to(tt.device("cuda:0"))
    label_batch_ = tokenizer(label_batch, return_tensors="pt", padding=True)["input_ids"]

    input_length = input_batch_.shape[-1]
    output_length = label_batch_.shape[-1]
    
    with tt.no_grad():
        output_batch = model.generate(input_batch_, max_length=input_length + MAX_OUTPUT_LENGTH)
        output_batch = output_batch[:,input_length:]

    output = tokenizer.batch_decode(output_batch)
    del input_batch_
    del output_batch
    del label_batch_
    tt.cuda.empty_cache()

    output = cut_last_break(output)
    output = parse_options(output)

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict[str, Any]:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [8]:
BATCH_SIZE = 4
input_batch = tf_dataset_test["inp"][:BATCH_SIZE]
label_batch = tf_dataset_test["distractors"][:BATCH_SIZE]
rans_batch = tf_dataset_test["right_answer"][:BATCH_SIZE]

In [9]:
label_batch

['\n  Самые великие земледелцы пустыни - это люди.\n  Пустыни быстро растут.\n  Размеры пустыни постоянно меняются.',
 '\n  Нельсон Мандела не был его оригинальным именем.\n  Нельсон Мандела был назван своим учителем.\n  Нельсон Мандела основал свою собственную юридическую фирму до того, как получил степень юриста.',
 '\n  Детям нехорошо веселиться летом.\n  Детям будет скучно читать программы\n  Учителям не нужно помогать детям анализировать уроки.',
 '\n  Фестиваль Гластонбери работает на прибыльной основе.\n  Джеймс Браун и Джосс Стоун родились в бедных семьях.\n  В 1970 году на фестивале Гластонбери можно бесплатно пообедать на ферме.']

In [10]:
parse_options(label_batch)

['Пустыни быстро растут.\nРазмеры пустыни постоянно меняются.\nСамые великие земледелцы пустыни - это люди.',
 'Нельсон Мандела был назван своим учителем.\nНельсон Мандела не был его оригинальным именем.\nНельсон Мандела основал свою собственную юридическую фирму до того, как получил степень юриста.',
 'Детям будет скучно читать программы\nДетям нехорошо веселиться летом.\nУчителям не нужно помогать детям анализировать уроки.',
 'В 1970 году на фестивале Гластонбери можно бесплатно пообедать на ферме.\nДжеймс Браун и Джосс Стоун родились в бедных семьях.\nФестиваль Гластонбери работает на прибыльной основе.']

In [11]:
rans_batch

['В пустыне нет живых существ.',
 'Нельсон Мандела изучал этот закон без перерыва в течение 50 лет.',
 'Летние программы могут помочь детям.',
 'Билеты на Фестиваль Гластонбери 2004 года были очень востребованы, несмотря на высокую цену.']

In [12]:
output_batch = get_metric_inputs(input_batch, label_batch, model, tokenizer)

In [13]:
output_batch

['В конце концов, человек был создан.\nВ последние 100 лет на протяжении миллионов лет менялись большие и мягкие пустыни.\nЛюди могут видеть деревья на вершине холма.',
 'В 1954 году г-н Мандела создал первую черную юридическую компанию.\nВ течение 50 лет он изучал право только ночью.\nЕго спасла работа тамбовского адвоката.',
 'Дети могут читать в летние каникулы.\nДетям следует избегать телевидения во время летних каникул.\nНе следует поощрять родителей читать детям отрывки.',
 'В конце 70-х годов фестиваль был закрыт.\nФестиваль вГластонбери был популярен в течение долгого времени.\nФестиваль вГластонбери был популярен по-своему.']

In [14]:
BATCH_SIZE = 1
N_STEPS = (len(tf_dataset_val) // BATCH_SIZE) + 1

metrics_val = []

for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):
    slice = tf_dataset_val[i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    if slice["inp"]:
        if "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?" in slice["inp"][0]:
            question = "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?"
        else:
            question = "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "

        distractors = slice["distractors"]

        output = get_metric_inputs(slice["inp"], distractors, model, tokenizer)

        distractors = parse_options(distractors)

        metric = compute_metrics(output, distractors)

        # код далее подходит только для батчей из одиночных примеров (BATCH_SIZE=1):
        metrics_val.append({
            "article": slice["article_ru"][0],
            "right_answer": slice["right_answer"][0],
            "question": question,
            "distractors": distractors[0],
            "output": output[0],

            "bleu": metric["bleu"]["bleu"],
            "sbleu": metric["sbleu"]["score"],
            "rouge1": metric["rouge"]["rouge1"],
            "rouge2": metric["rouge"]["rouge2"],
            "rougeL": metric["rouge"]["rougeL"],
            "rougeLsum": metric["rouge"]["rougeLsum"],
            "meteor": metric["meteor"]["meteor"],

            "article_orig": slice["article"][0],
            "question_orig": slice["question"][0],
            "options_orig": slice["options"][0],
            "right_answer_orig": slice["answer"][0]
        })

/tmp/ipykernel_100942/2770236774.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):


  0%|          | 0/176 [00:00<?, ?it/s]

In [15]:
metrics_val = pd.DataFrame(metrics_val)

In [16]:
metrics_val.describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.053317,7.663902,0.018136,0.008000,0.018136,0.018136,0.189195
std,0.185416,17.930034,0.107681,0.081255,0.107681,0.107681,0.183661
min,0.000000,0.462819,0.000000,0.000000,0.000000,0.000000,0.041528
25%,0.000000,1.719079,0.000000,0.000000,0.000000,0.000000,0.094045
50%,0.000000,2.548808,0.000000,0.000000,0.000000,0.000000,0.128940
75%,0.000000,4.453373,0.000000,0.000000,0.000000,0.000000,0.193556
max,1.000000,100.000000,1.000000,1.000000,1.000000,1.000000,0.999995


In [17]:
metrics_val.to_excel("RuGPT3Metrics-TF-val.xlsx", engine="openpyxl")